# Week9 - Ensemble Assignment

- create a training and test set with random_state = 3
- create a pipeline to extract new features
- try bagging & boosting algorithms


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

names = 'https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/names/us_names.csv'

df = pd.read_csv(names)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27999 entries, 0 to 27999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    27999 non-null  object
 1   gender  27999 non-null  object
dtypes: object(2)
memory usage: 656.2+ KB


In [2]:
df

,name,gender
0,jerilynn,F
1,percy,M
2,landry,M
3,reynalda,F
4,girl,F
...,...,...
27995,emelio,M
27996,holt,M
27997,silvestre,M
27998,annastacia,F


In [3]:
X = df[['name']]
y = df.gender

create a pipeline to extract new features

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

class MyFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit( self, X, y = None ):
        return self
    
    def transform(self, X, y=None):
        X['first_letter'] = [name[0] for name in X['name']]
        X['last_letter'] = [name[-1] for name in X['name']]

        return X[['first_letter', 'last_letter']].values

pipe = Pipeline([ 
    ("feature_engineering", MyFeatures()),
#     ("selector_new", FeatureSelector(["daily_trend"])),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

Xmatrix = pipe.fit_transform(X, y)
Xmatrix

<27999x52 sparse matrix of type '<class 'numpy.float64'>'
	with 55998 stored elements in Compressed Sparse Row format>

Tried to gather various features like length but unable to get good accuracy compared to above one . 
I was able to get near to 94 accuracy with length .But using first letter and last letter able to get 95 accuracy .

In [5]:
pipe[1].get_feature_names()

array(['x0_a', 'x0_b', 'x0_c', 'x0_d', 'x0_e', 'x0_f', 'x0_g', 'x0_h',
       'x0_i', 'x0_j', 'x0_k', 'x0_l', 'x0_m', 'x0_n', 'x0_o', 'x0_p',
       'x0_q', 'x0_r', 'x0_s', 'x0_t', 'x0_u', 'x0_v', 'x0_w', 'x0_x',
       'x0_y', 'x0_z', 'x1_a', 'x1_b', 'x1_c', 'x1_d', 'x1_e', 'x1_f',
       'x1_g', 'x1_h', 'x1_i', 'x1_j', 'x1_k', 'x1_l', 'x1_m', 'x1_n',
       'x1_o', 'x1_p', 'x1_q', 'x1_r', 'x1_s', 'x1_t', 'x1_u', 'x1_v',
       'x1_w', 'x1_x', 'x1_y', 'x1_z'], dtype=object)

In [6]:
data = pd.DataFrame(Xmatrix.toarray(), columns=pipe[1].get_feature_names(), dtype=int)

In [7]:
data

,x0_a,x0_b,x0_c,x0_d,x0_e,x0_f,x0_g,x0_h,x0_i,x0_j,...,x1_q,x1_r,x1_s,x1_t,x1_u,x1_v,x1_w,x1_x,x1_y,x1_z
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27994,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27995,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


After creating dataframe from above code .we need to split the data for training and testing.
the size of dataframe is 27999* 52

create a training and test set with random_state = 3

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xmatrix, y, test_size=0.2, random_state=3)# as mentioned traiing and testing with random state - .3
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(22399, 52)
(22399,)
(5600, 52)
(5600,)


BAGGING

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


decision_tree_base = DecisionTreeClassifier(random_state=3)

param_grid = {
    'base_estimator__max_depth': [None, 10, 20, 30],
    'n_estimators': [50, 100, 150],
}# given above parameters to test best values for prediction 

# included bagging inside grdisearch 
grid_search = GridSearchCV(
    BaggingClassifier(base_estimator=decision_tree_base, random_state=3),# Bagging Classifier with Decision Tree base estimator
    param_grid,
    cv=3  # cross validation
)


# Fit the grid search to the training data to find the best hyperparameters
grid_search.fit(X_train, y_train)
best_bagging_classifier = grid_search.best_estimator_# Get the best Bagging Classifier with Decision Tree base estimator and its hyperparameters
best_params = grid_search.best_params_
bagging_dt_predictions = best_bagging_classifier.predict(X_test)# Make predictions on the test data using the best estimator
bagging_dt_accuracy = accuracy_score(y_test, bagging_dt_predictions)

print(f"Best Bagging with Decision Tree Hyperparameters: {best_params}")
print(f"Best Bagging with Decision Tree Accuracy: {bagging_dt_accuracy:.2f}")


Best Bagging with Decision Tree Hyperparameters: {'base_estimator__max_depth': 10, 'n_estimators': 150}
Best Bagging with Decision Tree Accuracy: 0.75


BOOSTING

In [13]:
from sklearn.ensemble import AdaBoostClassifier  # Import AdaBoostClassifier
# Boosting with AdaBoost
adaboost_classifier = AdaBoostClassifier(n_estimators=100, random_state=3)  # Create an AdaBoost classifier with 100 base estimators and a fixed random state
adaboost_classifier.fit(X_train, y_train)  # Train the AdaBoost classifier on the training data
adaboost_predictions = adaboost_classifier.predict(X_test)  # Make predictions on the test data using the trained AdaBoost classifier
adaboost_accuracy = accuracy_score(y_test, adaboost_predictions)  # Calculate the accuracy of the AdaBoost model on the test data

# Print the accuracy of the AdaBoost model
print(f"AdaBoost Accuracy: {adaboost_accuracy:.2f}")


AdaBoost Accuracy: 0.75
